Класс API WB, указан ниже
Его необходимо выгрузить в отдельный файл на сервер
При необходимости в него будут добавляться новые функции

In [1]:
import datetime
import requests
import time
import json
import pandas as pd
from datetime import datetime
from datetime import timedelta

class Meta:
    def __init__(self):
        self._token = ''
        
        self._requests_api_stock='https://suppliers-stats.wildberries.ru/api/v1/supplier/stocks?dateFrom=2020-04-26T10%3A00%3A00.000Z&key='
        self._requests_api_orders= 'https://suppliers-stats.wildberries.ru/api/v1/supplier/orders?dateFrom='
        self._requests_api_meta2 = 'T21%3A00%3A01.000Z&flag=1&key='
        self._requests_api_meta3 = 'T21%3A00%3A01.000Z&flag=0&key='
        
        self._requests_api_sales= 'https://suppliers-stats.wildberries.ru/api/v1/supplier/sales?dateFrom='
        
        
        
class Stock(Meta):
    def get_stock(self):
        response_stock = requests.get(self._requests_api_stock + self._token, stream=True)
        df = pd.DataFrame(response_stock.json())
        now = datetime.now()
        return df
        
class Orders(Meta):
    def get_orders(self,date):
        """подставить date в формате 2021-01-01"""
        response_orders = requests.get(self._requests_api_orders+date+self._requests_api_meta2+ self._token, stream=True)
        df = pd.DataFrame(response_orders.json())
        return df   
        
    def get_orders_last_day(self):
        """возвращает заказы предыдущего дня"""
        date = (datetime.now()-timedelta(1)).strftime("%Y-%m-%d")
        return self.get_orders(date)
        
    def get_all_orders(self,date):
        """подставить date в формате 2021-01-01"""
        response_orders = requests.get(self._requests_api_orders+date+self._requests_api_meta3+ self._token, stream=True)
        df = pd.DataFrame(response_orders.json())
        return df   
        
class Sales(Meta):
    def get_sales(self,date):
        """подставить date в формате 2021-01-01"""
        response_sales = requests.get(self._requests_api_sales+date+self._requests_api_meta2+ self._token, stream=True)
        print(response_sales.text)
        df = pd.DataFrame(response_sales.json())
        return df       
        
    def get_sales_last_day(self):
        """возвращает заказы предыдущего дня"""
        date = (datetime.now()-timedelta(1)).strftime("%Y-%m-%d")
        return self.get_sales(date)
        
    def get_all_sales(self,date):
        """подставить date в формате 2021-01-01"""
        response_sales = requests.get(self._requests_api_sales+date+self._requests_api_meta3+ self._token, stream=True)
        df = pd.DataFrame(response_sales.json())
        return df 
    

class Report_total(Meta):
    def get_report(self,date_start,date_finish):
        """подставить date в формате 2021-01-01"""
        import urllib
        url_report = 'https://suppliers-stats.wildberries.ru/api/v1/supplier/reportDetailByPeriod'
        
        df = pd.DataFrame()
        for i in range(1):
            time.sleep(10)
            params = {
                'dateFrom':date_start,
                'key': self._token,
                'limit':1000,
                'rrdid':i,
                'dateto':date_finish

            }
            print(i,end = ', ')
            url = url_report+"?"+urllib.parse.urlencode(params)
            
            response_sales = requests.get(url, stream=True)
            df_local = pd.DataFrame(response_sales.json())
            if df_local.size ==0:
                break
            
            df = pd.concat([df,df_local])
        return df
    
    
    
    

# Выгрузка и обработка датасетов

Количество дней выгрузки

In [3]:
days_range = 15

In [4]:
from datetime import datetime,timedelta
import time

start_date = datetime.now() - timedelta(days=days_range)
start_date = start_date.strftime('%Y-%m-%d')

In [5]:
 date_now = datetime.now().strftime('%Y-%m-%d')

In [6]:
start_date

'2022-07-21'

Выгружаем продажи. Если выгрузка идет в Jupyter то ячейку с API предварительно надо выполнить ячейку с классом API WB.

Если выгрузка идет с сервера, то надо дописать выгрузку

from Get_API_WB_DATA import Stock
from Get_API_WB_DATA import Orders
from Get_API_WB_DATA import Sales

Get_API_WB_DATA - название py файла в папке со скриптом

## Продажи WB

In [7]:
df_sales = Sales().get_all_sales(start_date)

In [8]:
df_sales.head(3)

,date,lastChangeDate,supplierArticle,techSize,barcode,totalPrice,discountPercent,isSupply,isRealization,promoCodeDiscount,...,finishedPrice,priceWithDisc,nmId,subject,category,brand,IsStorno,gNumber,sticker,srid
0,2022-07-21T20:39:08,2022-07-21T21:00:53,1020110201,0,4039184102015,3760,45,False,True,0,...,1866.82,2120.25,2075446,Круги для плавания,Спортивный товар,SWIMTRAINER,0,1658389607400535511,,
1,2022-07-21T20:36:34,2022-07-21T21:00:53,tws-strawcup78593,0,7350083125934,1085,27,False,True,0,...,697.00,792.05,10002633,Поильники,Товары для малышей,Twistshake,0,91668749302533629091,,
2,2022-07-21T20:39:30,2022-07-21T21:00:53,tws-strawcup78588,0,7350083125880,1085,27,False,True,0,...,697.00,792.05,10002629,Поильники,Товары для малышей,Twistshake,0,92052952138051820930,,


In [9]:
time.sleep(60) # если сразу данные выгружать, то выдает ошибку

Далее надо убедиться, чтобы колонки не добавлялись и скрипт работал стабильно
Для этого отберем из датасета стандартные колонки

In [10]:
standart_col_sale = ['date', 'lastChangeDate', 'supplierArticle', 'techSize', 'barcode',
       'totalPrice', 'discountPercent', 'isSupply', 'isRealization',
       'promoCodeDiscount', 'warehouseName', 'countryName', 'oblastOkrugName',
       'regionName', 'incomeID', 'saleID', 'odid', 'spp', 'forPay',
       'finishedPrice', 'priceWithDisc', 'nmId', 'subject', 'category',
       'brand', 'IsStorno', 'gNumber', 'sticker']

In [11]:
df_sales = df_sales[standart_col_sale]

## Заказы WB

In [12]:
df_order = Orders().get_all_orders(start_date)

In [13]:
df_order.tail(4)

,date,lastChangeDate,supplierArticle,techSize,barcode,totalPrice,discountPercent,warehouseName,oblast,incomeID,odid,nmId,subject,category,brand,isCancel,cancel_dt,gNumber,sticker,srid
7010,2022-08-04T23:51:12,2022-08-05T06:54:26,alilo-R160908,0,6954644609089,4235.0,25,Санкт-Петербург 2,Псковская,7866200,600251615321,8694822,Игрушки интерактивные,Игрушки,Alilo,False,0001-01-01T00:00:00,6096230152841304041,,3c72b5a4aa8640fca3bca54014c93687
7011,2022-08-04T23:51:30,2022-08-05T06:54:26,alilo-R160905,0,6954644609058,4235.0,25,Коледино,Ростовская,8258614,600251616220,8694820,Игрушки интерактивные,Игрушки,Alilo,False,0001-01-01T00:00:00,4617846645260307759,,47a2fdb262c745da94e6c7bc2e9e4aee
7012,2022-08-04T23:53:43,2022-08-05T06:54:26,tws-bottle18078249,0,7350083122490,1105.0,33,Санкт-Петербург 2,Карелия,8334351,600251623833,9242187,Бутылочки для кормления,Товары для малышей,Twistshake,False,0001-01-01T00:00:00,8103674893549358226,,b5684eb52bea4685b552ae6212fb3e92
7013,2022-08-04T23:56:23,2022-08-05T06:54:26,alilo-R160908,0,6954644609089,4235.0,25,Коледино,Краснодарский,8258614,600251632922,8694822,Игрушки интерактивные,Игрушки,Alilo,False,0001-01-01T00:00:00,98299114283244096874,,448773a3665543629b488c73de2c7231


In [14]:
time.sleep(60)

In [15]:
standart_col_order = ['date', 'lastChangeDate', 'supplierArticle', 'techSize', 'barcode',
       'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'incomeID',
       'odid', 'nmId', 'subject', 'category', 'brand', 'isCancel', 'cancel_dt',
       'gNumber', 'sticker']

In [16]:
df_order = df_order[standart_col_order]

# Остатки WB

In [17]:
df_stock = Stock().get_stock()

In [18]:
df_stock.loc[:,'Date'] = date_now

In [19]:
df_stock.head(3)

,lastChangeDate,supplierArticle,techSize,barcode,quantity,isSupply,isRealization,quantityFull,quantityNotInOrders,warehouse,...,inWayFromClient,nmId,subject,category,daysOnSite,brand,SCCode,Price,Discount,Date
0,2021-12-31T02:49:31.163,ZA-BO-01ZA-BO-01,0,0703625107115,0,False,True,1,0,6,...,0,6406795,Ночники,Дом,1184,ZAZU,3100-8906,3685,35,2022-08-05
1,2021-12-31T02:49:31.163,ltd-bag1-liteL10_02_030_GRY,S 3.5-6 кг,2001573374008,0,False,True,1,0,3,...,0,21674409,Конверты для пеленания,Товары для малышей,273,Love To Dream,3100-8906,1845,8,2022-08-05
2,2022-07-08T04:29:27.043,bab-avi3NAV-008,3-7лет,0857788003071,0,False,True,1,1,1,...,0,2075466,Солнцезащитные очки,Аксессуары,0,Babiators,3100-8906,3430,20,2022-08-05


In [20]:
standart_col_stock = ['lastChangeDate', 'supplierArticle', 'techSize', 'barcode', 'quantity',
       'isSupply', 'isRealization', 'quantityFull', 'quantityNotInOrders',
       'warehouse', 'warehouseName', 'inWayToClient', 'inWayFromClient',
       'nmId', 'subject', 'category', 'daysOnSite', 'brand', 'SCCode', 'Price',
       'Discount', 'Date']

In [21]:
df_stock = df_stock[standart_col_stock]

# Обработка данных 

In [22]:
df_order = df_order.astype(
    {'incomeID': 'str', 
     'odid': 'str',
     'nmId': 'str'})

df_sales = df_sales.astype(
    {'incomeID': 'str', 
     'odid': 'str',
     'nmId': 'str'})

df_stock =df_stock.astype({'nmId': 'str'})

In [23]:
# выгрузка датафрэйма с проблемными баркодами
path_wb_ms = '/Users/Aleksey/Documents/Файлы для БД/WB_MS.xlsx'
df_ms_wb_code = pd.read_excel(path_wb_ms,dtype = {'nmId': 'str', 
     'barcode': 'str',
     'barcode_ms': 'str'})

df_ms_wb_code = df_ms_wb_code[['nmId','barcode_ms']]

In [24]:
def set_code(data):
    data = data.merge(right = df_ms_wb_code,how = 'left', 
                          on = 'nmId',suffixes = ('_wb','_wb_2'))
    data.loc[data['barcode_ms'].isnull(),'barcode_ms'] = data.loc[data['barcode_ms'].isnull(),'barcode']
    return data

In [25]:
df_order = set_code(df_order)

In [26]:
df_stock = set_code(df_stock)

In [27]:
df_sales = set_code(df_sales)

In [28]:
for datadrame_wb in [df_order,df_stock,df_sales]:
    if datadrame_wb[datadrame_wb.barcode_ms == ""].size.sum()>0:
        print('error')
    else:
        print('ok',end = ', ')

error
error
ok, 

# Коннект с базой данных

In [29]:
import sqlalchemy
import pandas as pd

database_username = 'alex'
database_password = 'dnr83jfy73Bv3hjjJ'
database_ip       = '82.146.50.47'
database_name     = 'm2c'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [30]:
name_main_table_order = 'wb_order'
name_main_table_sale = 'wb_sale'
name_main_table_stock = 'wb_stock'

Для начала необходимо создать order таблицу, затем при помощи JOIN объединить их по уникальному ID
Для таблиц order и sales это "odid"

In [31]:
def join_table(name_table,data):
    name_temp = name_table+'_temp'
    for i in range(0,len(data),50):
        with database_connection.begin() as connection:
            data.iloc[i:i+50].to_sql(con=connection, name=name_temp, if_exists='append')
            if i%1000==0:
                print(i, end=', ')
                
    SQL_DROP = f"""DELETE FROM {name_table} 
    where {name_table}.odid in (select odid from {name_temp})"""
    
    result = database_connection.execute(SQL_DROP)
    result = database_connection.execute(f"INSERT INTO {name_table} SELECT * FROM {name_temp}")
    result = database_connection.execute(f"DROP TABLE {name_temp}")
    
    
def join_stock(name_table, data):
    name_temp = name_table+'_temp'
    for i in range(0,len(data),50):
        with database_connection.begin() as connection:
            data.iloc[i:i+50].to_sql(con=connection, name=name_temp, if_exists='append')
            if i%1000==0:
                print(i, end=', ')
                
    SQL_DROP = f"""DELETE FROM {name_table} 
    where {name_table}.Date = '{date_now}'"""
    
    result = database_connection.execute(SQL_DROP)
    result = database_connection.execute(f"INSERT INTO {name_table} SELECT * FROM {name_temp}")
    result = database_connection.execute(f"DROP TABLE {name_temp}")

In [32]:
join_table(name_main_table_order,df_order)

0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 

In [33]:
join_table(name_main_table_sale,df_sales)

0, 1000, 2000, 3000, 4000, 5000, 6000, 

In [34]:
join_stock(name_main_table_stock,df_stock)

0, 1000, 2000, 

In [35]:
database_connection.dispose()

Вариант когда нету таблицы

In [29]:
# def insert_table(name_table,data):
#     for i in range(0,len(data),50):
#         with database_connection.begin() as connection:
#             data.iloc[i:i+50].to_sql(con=connection, name=name_table, if_exists='append')
#             if i%1000==0:
#                 print(i, end=', ')

In [30]:
# insert_table(name_main_table_order,df_order)

0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 

In [31]:
# insert_table(name_main_table_sale,df_sales)

0, 1000, 2000, 3000, 4000, 5000, 6000, 

In [32]:
# insert_table(name_main_table_stock,df_stock)

0, 1000, 2000, 